In [12]:
!pip install -q --upgrade pip
!pip install -q --upgrade diffusers transformers scipy ftfy huggingface_hub accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 2.3 MB/s eta 0:00:00


In [13]:
from huggingface_hub import notebook_login

# Required to get access to stable diffusion model
notebook_login()

In [14]:
!nvidia-smi

Sat Feb 10 17:06:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Steps

1. Import “torch” and the Hugging Face StableDiffusionPipeline class from the diffusers library;
2. Define the stable-diffusion-v1-5 model to pull in and;
3. Specify fp16 for speed and ensure we are using a GPU.

In [15]:
import torch
from diffusers import StableDiffusionPipeline

In [11]:
model_id = "runwayml/stable-diffusion-v1-5"
pipeline = StableDiffusionPipeline.from_pretrained(
    pretrained_model_name_or_path = model_id,
    torch_dtype = torch.float32, # change to flaot 32 for CPU
    revision = "fp16"

)

/usr/local/lib/python3.10/dist-packages/diffusers/pipelines/pipeline_utils.py:270: FutureWarning: You are loading the variant fp16 from runwayml/stable-diffusion-v1-5 via `revision='fp16'` even though you can load it via `variant=`fp16`. Loading model variants via `revision='fp16'` is deprecated and will be removed in diffusers v1. Please use `variant='fp16'` instead.
  warnings.warn(
text_encoder/model.safetensors not found
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


# We will create our function which defines our parameters when we generate our images.

In [16]:
import os

from IPython.display import Image, display

The guidance scale is defining the level of freedom to give the model when generating images. The lower the number, the more freedom your model has when generating an image.

In [17]:
def generate_images(
    prompt,
    num_images_to_generate,
    num_images_per_prompt = 4,
    guidance_scale = 8,
    output_dir = "images",
    display_images = False):

  # number of times diffusion pipeline should iterate to create n images
  num_iterations = num_images_to_generate // num_images_per_prompt
  os.makedirs(output_dir, exist_ok = True)

  for i in range(num_iterations):
    # pipeline
    images = pipeline(
        prompt,
        num_images_per_prompt= num_images_per_prompt,
        guidance_scale= guidance_scale,

    )

    # Save generated image
    for idx, image in enumerate(images.images):
      image_name = f"{output_dir}/image_{ (i * num_images_per_prompt) + idx}.png"
      image.save(image_name)

      if display_images:
        display(Image(filename = image_name, width = 128, height = 128))

In [ ]:
# 1000 images takes a couple hours on a SageMaker Studio Lab GPU instance

generate_images("human on a forklift", num_images_to_generate = 4, guidance_scale=4, display_images=True)

  0%|          | 0/50 [00:00<?, ?it/s]